__Obiettivo__

Migliorare l'estrazione delle sezioni che compongono il PDF, concentrandosi soprattutto su _Abstract_ ed _Introduction_, su cui saranno applicate tecniche per estrapolare le _keywork_. Pertanto occorre ripetere l'analisi affrontata nel file _metadata.ipynb_, ma escludendo alcune sezioni e concentrandosi solamente su alcune di esse.

Potrebbe essere adeguato nuovamente l'approccio delineato in _metadata.ipynb_, dove è individuato l'indice della linea, successivamente ad un'azione di _splitlines_, in cui compaia la _keyword_.

La pipeline si suddivide in:
- __Regex__, individuare i pattern principali per riconoscere alcune parole / frasi ripetitive nel testo
- __Estrazione__, estrapolazione della sezione di testo coordinata rispetto allo _start index_ ed _end index_

In [1]:
pdf_path = "../articles/"

In [2]:
class ScannedText:
    def __init__(self, start_index, end_index, text):
        self.start_index = start_index
        self.end_index = end_index
        self.text = text

In [3]:
import os

from typing import List

def get_path_pdf_files(pdf_path: str) -> List[str]:
    path_files = []

    for path in os.listdir(pdf_path):
        path_files.append(pdf_path + path)
    
    return path_files

path_pdf_files = get_path_pdf_files(pdf_path)

In [4]:
import pytesseract

from typing import Dict
from pdf2image import convert_from_path

# It will convert all the pages to images
def convert_pdf_to_images(path: str) -> list:
    try:
        return convert_from_path(path)
    except Exception as e:
        print("Error occur during conversion from pdf to image:", e)
        return None

def convert_pdf_to_text(paths: list[str]) -> Dict[str, str]:
    _dict: Dict[str, str] = {}

    for path in paths:
        images = convert_pdf_to_images(path)    
        
        text = ""
        for image in images:
            try:
                text = text + pytesseract.image_to_string(image)
            except Exception as e:
                print("Error occur during conversion from image to string:", e)
        
        _dict[path] = text

    return _dict

dict_scanned_text = convert_pdf_to_text(path_pdf_files)

Fino ad ora è stato riportato lo stesso codice utilizzato in _metadata.ipynb_ per la conversione del file PDF in _str_, affinchè sia possibile attuare tecniche di estrazione del testo con maggiore facilità, oltre alla compatibilità garantita mediante l'impiego di un linguaggio di programmazione.

_Nota bene_: nella funzione *convert_pdf_to_images* è applicata la conversione di ciascuna pagina che componga il PDF, a differenza di _metadata.ipynb_ in cui sono convertite solamente le facciate iniziali dato che l'obiettivo ricade nella definizione dei metadati di ogni file.

In [12]:
import re

from typing import Dict
from itertools import islice

def get_target_line_index(expression: str, text: str) -> int:
    count_line = 0
    lines = text.splitlines()

    for line in lines:
        if len(re.findall(expression, line.lower())) > 0:
            return count_line
        
        count_line += 1

    return -1

def remove_references_section(expression: str, dict_scanned_text: Dict[str, str]) -> Dict[str, str]:
    _dict: Dict[str, str] = {}

    for key, value in dict_scanned_text.items():
        index = get_target_line_index(expression, value.lower())

        if index > -1:
            lines = value.splitlines()

            text = ""
            for line in islice(lines, 0, index):
                text += line + "\n"
                _dict[key] = text

    return _dict

dict_scanned_text_without_references = remove_references_section(r"^[0-9]?.?\s*\breferences\b$", dict_scanned_text)

Sembra che non abbiano la sezione _References_ i seguenti file:
- _83CondonThompson_
- _07Beal_
- _76Panek_

Dopo un breve controllo, i file stessi non possiedono la sezione al loro interno.

In [13]:
def get_section_scanned_text(start_word:str, end_word:str, dict_scanned_text: dict[str, str]) -> Dict[str, ScannedText]:
    _dict : Dict[str, ScannedText] = {}

    for key, value in dict_scanned_text.items():
        _dict[key] = ScannedText(get_target_line_index(start_word, value), get_target_line_index(end_word, value), value)

    return _dict

dict_scanned_abstract = get_section_scanned_text(r"^abstract", r"1?\.?\s*introduction", dict_scanned_text_without_references)
dict_scanned_introduction = get_section_scanned_text(r"^1?\.?\s*introduction", r"^2.?\W", dict_scanned_text_without_references)

def stamp_found_indexes(_dict: Dict[str, ScannedText]):
    for key, value in _dict.items():
        print(key + ": " + "(start_" + str(value.start_index) + ", end_" +  str(value.end_index) + ")")

stamp_found_indexes(dict_scanned_abstract)

print("\n")

stamp_found_indexes(dict_scanned_introduction)

../articles/16DavidNetanyahuWolf.pdf: (start_21, end_35)
../articles/90GeorgeSchaeffer.pdf: (start_10, end_30)
../articles/91FeldmannMysliwietzMonien.pdf: (start_8, end_20)
../articles/09CiancariniFavini 1.pdf: (start_7, end_40)
../articles/ICGA_J_34_2_HHB_Zugzwangs_in_Chess_Studies.pdf: (start_41, end_51)
../articles/19Kamlish.pdf: (start_10, end_39)
../articles/96Brockington.pdf: (start_-1, end_5)


../articles/16DavidNetanyahuWolf.pdf: (start_35, end_58)
../articles/90GeorgeSchaeffer.pdf: (start_30, end_99)
../articles/91FeldmannMysliwietzMonien.pdf: (start_20, end_105)
../articles/09CiancariniFavini 1.pdf: (start_40, end_99)
../articles/ICGA_J_34_2_HHB_Zugzwangs_in_Chess_Studies.pdf: (start_51, end_101)
../articles/19Kamlish.pdf: (start_39, end_603)
../articles/96Brockington.pdf: (start_5, end_30)


In [ ]:
import time

# Check if the found lines are correct according to the extraction
for key in dict_scanned_text_without_references.keys():
    with open("../txt/with.txt", "w") as file:
        file.write(dict_scanned_text[key])

    time.sleep(15)

    with open("../txt/without.txt", "w") as file:
        file.write(dict_scanned_text_without_references[key])

    time.sleep(15)

Rispetto alla keyword _1. Introduction_, alcuni file presentano delle limitazioni, quali:
- _83CondonThompson_, scan scadente
- _76Panek_, assente la sezione _Introduction_
- _19Kamlish_, disposizione del documento in due colonne distinte, come già anticipato, rende più ardua la conversione da _images_ a _text_, poichè sovrappone il contenuto disposto tra le due colonne. Ciò avviene anche per il file _09CiancariniFavini_.